<a href="https://colab.research.google.com/github/atlimited/llm_train/blob/main/deepseek.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
!cp drive/MyDrive/train.txt .

In [3]:
!GIT_LFS_SKIP_SMUDGE=1 git clone https://huggingface.co/deepseek-ai/deepseek-moe-16b-base

Cloning into 'deepseek-moe-16b-base'...
remote: Enumerating objects: 40, done.
remote: Counting objects: 100% (36/36), done.
remote: Compressing objects: 100% (36/36), done.
remote: Total 40 (delta 12), reused 0 (delta 0), pack-reused 4
Unpacking objects: 100% (40/40), 1.38 MiB | 3.92 MiB/s, done.


In [4]:
!mkdir deepseek
!cp deepseek-moe-16b-base/*.json deepseek/
!cp deepseek-moe-16b-base/*.py deepseek/

In [5]:
!ls -lh deepseek

total 5.0M
-rw-r--r-- 1 root root 1.1K Apr 22 05:24 config.json
-rw-r--r-- 1 root root  10K Apr 22 05:24 configuration_deepseek.py
-rw-r--r-- 1 root root  121 Apr 22 05:24 generation_config.json
-rw-r--r-- 1 root root  72K Apr 22 05:24 modeling_deepseek.py
-rw-r--r-- 1 root root 479K Apr 22 05:24 model.safetensors.index.json
-rw-r--r-- 1 root root  793 Apr 22 05:24 tokenizer_config.json
-rw-r--r-- 1 root root 4.4M Apr 22 05:24 tokenizer.json


In [6]:
!git clone https://github.com/huggingface/transformers.git -b v4.38.2

Cloning into 'transformers'...
remote: Enumerating objects: 196791, done.
remote: Counting objects: 100% (1915/1915), done.
remote: Compressing objects: 100% (951/951), done.
remote: Total 196791 (delta 1187), reused 1399 (delta 841), pack-reused 194876
Receiving objects: 100% (196791/196791), 210.18 MiB | 14.41 MiB/s, done.
Resolving deltas: 100% (139410/139410), done.
Note: switching to '092f1fdaa4224fdd88c616dc9678e6fcb37bfffd'.

You are in 'detached HEAD' state. You can look around, make experimental
changes and commit them, and you can discard any commits you make in this
state without impacting any branches by switching back to a branch.

If you want to create a new branch to retain commits you create, you may
do so (now or later) by using -c with the switch command. Example:

  git switch -c <new-branch-name>

Or undo this operation with:

  git switch -

Turn off this advice by setting config variable advice.detachedHead to false



In [7]:
!cp transformers/examples/pytorch/language-modeling/run_clm.py .

In [8]:
!ls

deepseek  deepseek-moe-16b-base  drive	run_clm.py  sample_data  train.txt  transformers


In [9]:
!pip install -r transformers/examples/pytorch/language-modeling/requirements.txt

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 297.6/297.6 kB 9.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.0/542.0 kB 40.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 13.9 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-many

In [10]:
!pip install flash-attn --no-build-isolation

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 39.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.6/44.6 kB 7.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 307.2/307.2 kB 39.8 MB/s eta 0:00:00
  Created wheel for flash-attn: filename=flash_attn-2.5.7-cp310-cp310-linux_x86_64.whl size=120853563 sha256=bbe6f77fd0899f8a125a5bdcf734b660c4c88e81c9b51c7ce98ebeba44dc6fa0
  Stored in directory: /root/.cache/pip/wheels/13/96/ed/bcac89c56b606421f99b45b16a94db5d0f2b6b4eaf8bac4d01
Successfully built flash-attn


In [11]:
!cat /content/deepseek/config.json

{
  "architectures": [
    "DeepseekForCausalLM"
  ],
  "attention_bias": false,
  "attention_dropout": 0.0,
  "auto_map": {
    "AutoConfig": "configuration_deepseek.DeepseekConfig",
    "AutoModel": "modeling_deepseek.DeepseekModel",
    "AutoModelForCausalLM": "modeling_deepseek.DeepseekForCausalLM"
  },
  "bos_token_id": 100000,
  "eos_token_id": 100001,
  "first_k_dense_replace": 1,
  "hidden_act": "silu",
  "hidden_size": 2048,
  "initializer_range": 0.02,
  "intermediate_size": 10944,
  "max_position_embeddings": 4096,
  "model_type": "deepseek",
  "moe_intermediate_size": 1408,
  "moe_layer_freq": 1,
  "n_routed_experts": 64,
  "n_shared_experts": 2,
  "norm_topk_prob": false,
  "num_attention_heads": 16,
  "num_experts_per_tok": 6,
  "num_hidden_layers": 28,
  "num_key_value_heads": 16,
  "pretraining_tp": 1,
  "rms_norm_eps": 1e-06,
  "rope_scaling": null,
  "rope_theta": 10000,
  "scoring_func": "softmax",
  "tie_word_embeddings": false,
  "torch_dtype": "bfloat16",
  "trans

In [12]:
%%writefile /content/deepseek/config.json

{
  "architectures": [
    "DeepseekForCausalLM"
  ],
  "attention_bias": false,
  "attention_dropout": 0.0,
  "auto_map": {
    "AutoConfig": "configuration_deepseek.DeepseekConfig",
    "AutoModel": "modeling_deepseek.DeepseekModel",
    "AutoModelForCausalLM": "modeling_deepseek.DeepseekForCausalLM"
  },
  "bos_token_id": 100000,
  "eos_token_id": 100001,
  "first_k_dense_replace": 1,
  "hidden_act": "silu",
  "hidden_size": 512,
  "initializer_range": 0.02,
  "intermediate_size": 10944,
  "max_position_embeddings": 4096,
  "model_type": "deepseek",
  "moe_intermediate_size": 1408,
  "moe_layer_freq": 1,
  "n_routed_experts": 8,
  "n_shared_experts": 2,
  "norm_topk_prob": false,
  "num_attention_heads": 16,
  "num_experts_per_tok": 6,
  "num_hidden_layers": 14,
  "num_key_value_heads": 16,
  "pretraining_tp": 1,
  "rms_norm_eps": 1e-06,
  "rope_scaling": null,
  "rope_theta": 10000,
  "scoring_func": "softmax",
  "tie_word_embeddings": false,
  "torch_dtype": "bfloat16",
  "transformers_version": "4.36.0",
  "use_cache": true,
  "vocab_size": 102400
}

Overwriting /content/deepseek/config.json


In [13]:
import json
from deepseek.modeling_deepseek import DeepseekForCausalLM
from deepseek.configuration_deepseek import DeepseekConfig

model_dir = 'deepseek'
config_file = './deepseek/config.json'
with open(config_file, 'r') as f:
    config = json.load(f)
    config = DeepseekConfig.from_dict(config)

model = DeepseekForCausalLM(config)

model.save_pretrained(model_dir)

In [14]:
!cp train.txt ori.txt
!head -n 100 ori.txt > train.txt
!tail -n 100 ori.txt > val.txt

In [15]:
!python run_clm.py \
    --model_name_or_path=./deepseek \
    --train_file=train.txt \
    --validation_file=val.txt \
    --do_train \
    --do_eval \
    --num_train_epochs=30 \
    --save_steps=10000 \
    --save_total_limit=3 \
    --per_device_train_batch_size=1 \
    --per_device_eval_batch_size=1 \
    --output_dir=output/ \
    --overwrite_output_dir=true \
    --use_fast_tokenizer=False \
    --trust_remote_code=true

2024-04-22 05:25:58.534181: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-04-22 05:25:58.534240: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-04-22 05:25:58.535553: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-04-22 05:25:59.654613: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
04/22/2024 05:26:03 - WARNING - __main__ - Process rank: 0, device: cuda:0, n_gpu: 1, distributed training: False, 16-bits training: False
04/22/2024 05:26:03 - INFO - __main__ - Training/evaluation parameters TrainingArguments(
_n_gpu=1,
ac